# Was wordt lijst

Hoofdvraag: welke landelijke typologie (SBB-code, VvN of rVvN) wordt onze standaard die we als input voor Veg2Hab gebruiken en waarom?
- rVvN is het nieuwst, en beschrijft als het goed is de meeste vegetatietypen. gecheckt: als iets een SBB of VvN code heeft, heeft het ook altijd een rVvN code. Andersom niet.
- De opdrachtgever gaf als commentaar terug dat zij vinden dat VvN de standaard moet zijn.

#### Openstaande vragen voor Johannes / begeleidingscommissie:
- Waarom bevat kolom `rVvN` waardes 'Niet'? Moeten we die op NA zetten?
- Waarom staan sommige waardes in kolom `VvN` tussen haakjes? 
- Waarom wil opdrachtgever dat VvN de standaard input voor Veg2Hab is, en niet het nieuwere rVvN?
- Vanaf rij 989 bevat VvN geen waardes meer. rVvN wel, en sommige ook een SBB-code. Waarom geen VvN?
- Wat is de 17e kolom?

#### Beantwoorde vragen:

**Vraag**:

**Antwoord:**

## Was-wordt lijst inladen

In [112]:
import pandas as pd
import re

In [113]:
df = pd.read_excel('../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx')

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [114]:
ww = df[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype', 'SBB-code':'SBB'})

In [115]:
ww_not_H0000 = ww.__deepcopy__()

# Verwijderen van alle rijen uit de waswordt lijst die vertalen naar H0000. Deze verwacht je niet terug te vinden in de definitietabel
ww_not_H0000 = ww_not_H0000[ww_not_H0000['Habtype']!='H0000']
ww_not_H0000 = ww_not_H0000[~ww_not_H0000['Habtype'].isna()]

# Verwijderen van alle rijen in de waswordt lijst waarin voor het habtype verwezen wordt naar subassociaties
ww_not_H0000 = ww_not_H0000[~ww_not_H0000['Habtype'].apply(lambda x: 'zie subassociaties' in x if x is not None else False)]

# Nieuwe kolommen voor opgeschoonde SBB-code en VvN-code
ww_not_H0000['SBB-code'] = ww_not_H0000['SBB']
ww_not_H0000['VvN-code'] = ww_not_H0000['VvN']

# Opschonen van lege waardes naar None values
ww_not_H0000['SBB-code'][ww_not_H0000['SBB-code'].isna()] = None
ww_not_H0000['VvN-code'][ww_not_H0000['VvN-code'].isna()] = None

## Volledigheid typologie was-wordt

rVvN lijkt de meest specifieke vegetatietypologie. ALles met een waarde voor SBB of VvN heeft altijd (1 uitzondering, zie beneden) ook een rVvN code, andersom niet. 

Velden met een SBB code hebben altijd ook een habitattype ingevuld, velden met VvN of rVvN niet.

In [119]:
ww.isna().sum()

rVvN         5
VvN        257
SBB        352
Habtype    211
dtype: int64

In [120]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN         0
VvN        252
SBB        348
Habtype    207
dtype: int64

In [7]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN          0
VvN           0
SBB-code    224
Habtype     183
dtype: int64

In [8]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB-code'].isna()].isna().sum()

rVvN          1
VvN         129
SBB-code      0
Habtype       0
dtype: int64

In [9]:
ww[~ww['SBB-code'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB-code,Habtype
1216,NaN,NaN,41A3d,H0000


In [10]:
ww[ww.duplicated(keep=False)]

,rVvN,VvN,SBB-code,Habtype
1240,NaN,NaN,NaN,NaN
1241,NaN,NaN,NaN,NaN
1242,NaN,NaN,NaN,NaN
1243,NaN,NaN,NaN,NaN


#### SBB naar VvN vertalen en vice versa
Is er een veel-op-1 relatie tussen SBB en VvN?

In [121]:
number_of_VvN_per_SBB = ww_not_H0000.groupby('SBB-code')['VvN-code'].nunique()

In [122]:
number_of_VvN_per_SBB.value_counts()

VvN-code
1    442
0     75
3     12
2     11
4      6
5      2
Name: count, dtype: int64

In [123]:
number_of_VvN_per_SBB[number_of_VvN_per_SBB == 5]

SBB-code
07A-a    5
33A5     5
Name: VvN-code, dtype: int64

In [124]:
number_of_SBB_per_VvN = ww_not_H0000.groupby('VvN-code')['SBB-code'].nunique()

In [125]:
number_of_SBB_per_VvN.value_counts()

SBB-code
1    349
2     55
0     18
3     13
5      4
4      4
Name: count, dtype: int64

In [126]:
number_of_SBB_per_VvN[number_of_SBB_per_VvN == 5]

VvN-code
08Bd01    5
11Aa02    5
16aB01    5
43Ab1d    5
Name: SBB-code, dtype: int64

## Matchen was-wordt en definitietabel

Vinden we alle waarden voor rVvN, VvN en SBB uit de was-wordt lijst terug in de definitietabel? Alle vegetatietypen die niet automatisch tot H0000 leiden in de was wordt, zouden we willen terugvinden in de definietabel habitattypen.

Om deze twee tabellen goed te matchen, moeten we in ieder geval de volgende dingen doen:

- vegetatiecodes ontdoen van hoofdletters
- vegetatiecodes ontdoen van `leading zero's`
- VvN codes uit de was-wordt ontdubbelen
- VvN codes uit de was-wordt ontdoen van haakjes
- Codes voor rompgemeenschappen en derivaatgemeenschappen in de definitietabel ontdoen van extra formatting
- De kolom `Code vegetatietype` uit de definitietabel splitsen in twee kolommen voor VvN en SBB. 

In [127]:
dt = pd.read_excel('../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls', sheet_name=1)

#### Opschonen vegcodes

In [128]:
# Verwijderen van kolommen zonder vegetatiecode. Dit zijn kolommen met waterplantengemeenschappen en vegetatieloze regels.
dt = dt[~dt['Code vegetatietype'].isna()]

# Opsplitsen van kolom `Code vegetatietype` aparte kolommen voor SBB- en VvN-code
dt['SBB-code'] = dt['Code vegetatietype']
dt = dt.rename(columns={'Code vegetatietype':'VvN-code'})
dt['VvN-code'][dt['VvN-code'].apply(lambda x: 'SBB' in x)] = None
dt['SBB-code'][~dt['SBB-code'].apply(lambda x: 'SBB' in x)] = None

# Verwijderen van leading 'SBB-' in SBB-code.
dt['SBB-code'] = dt['SBB-code'].apply(lambda x: re.sub('SBB-', '', x) if x is not None else None)

In [129]:
def disect_vvn(vvn_code: str):
    # klasse
    klasse = None
    try:
        klasse_span = re.search(r'^\d+', vvn_code).span()
        klasse = vvn_code[klasse_span[0]:klasse_span[1]]
    except:
        print('klasse in VvN code not well defined')
    
    # orde en verbond
    orde = None
    try:
        orde = re.findall('[a-zA-Z]+', vvn_code)[0]
    except:
        print('orde in VvN code not well defined')
    
    # associatie
    associatie = None
    try:
        associatie = re.findall('\d+', vvn_code)[1]
    except:
        print('associatie in VvN code not well defined')
    
    # subassociatie
    subassociatie = None
    try:
        subassociatie = re.findall('[a-zA-Z]+', vvn_code)[1]
    except:
        print('subassociatie in VvN code not well defined')
    
    return klasse, orde, associatie, subassociatie


def clean_vegcode(vegcode: str):
    k, o, a, s = disect_vvn(vegcode)

    # verwijder leading zeros uit klasse 
    k = str(int(k))
    
    # orde en verbond naar lowercase
    o = o.lower()
    
    # verwijder leading zeros uit associatie
    if a is None:
        a = ''
    else:
        a = str(int(a))
    
    # als subassociatie niet aanwezig is, maak er een lege string van.
    if s is None:
        s = ''
    s = s.lower()
        
    return k + o + a + s


def convert_dg_rg_codes(vegcode: str):
    '''
    Alleen gebruikt voor de definitietabel habitattypen. Zet codes voor Rompgemeenschappen en Derivaatgemeenschappen om in normale vegcodes, zodat de waardes matchen met die in de waswordt lijst.
    '''
    return re.sub('-', '', re.sub('\[[\d\w]*\]', '', vegcode)) if vegcode is not None else None

    
def transform_vegcode(x):
    try:
        return clean_vegcode(x)
    except:
        return None

In [130]:
ww_not_H0000['VvN-code'] = ww_not_H0000['VvN-code'].apply(lambda x: transform_vegcode(x))
dt['VvN-code'] = dt['VvN-code'].apply(lambda x: transform_vegcode(x))

klasse in VvN code not well defined
orde in VvN code not well defined
associatie in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well 

In [131]:
ww_not_H0000['SBB-code'] = ww_not_H0000['SBB-code'].apply(lambda x: x.lower() if x is not None else None)
dt['SBB-code'] = dt['SBB-code'].apply(lambda x: x.lower() if x is not None else None)

In [132]:
dt['VvN-code'] = dt['VvN-code'].apply(lambda x: convert_dg_rg_codes(x))

In [133]:
ww_not_H0000.to_excel('../data/ww_cleaned.xlsx', index=False)
dt.to_excel('../data/dt_cleaned.xlsx', index=False)

#### Overlap rVvN en definitietabel

Geen enkele rVvN waarde wordt teruggevonden in de definitietabel. Dat klopt, want die zou alleen VvN en SBB moeten hanteren. 

In [98]:
# Aantal unieke waardes van rVvN.
len(set(ww_not_H0000['rVvN']))

561

In [99]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww_not_H0000['rVvN']).intersection(set(dt['VvN-code'])))

0

#### Overlap VvN en definitietabel

In [100]:
# Aantal unieke waardes van VvN in de was-wordt.
len(set(ww_not_H0000['VvN']))

434

In [101]:
# Aantal unieke waardes van VvN in de definitietabel.
len(set(dt['VvN-code']))

290

In [102]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww_not_H0000['VvN']).intersection(set(dt['VvN-code'])))

237

In [105]:
# VvN codes in de definitietabel die niet voorkomen in de was-wordt lijst
set(dt['VvN-code']) - (set(ww_not_H0000['VvN']).intersection(set(dt['VvN-code'])))

{'10aa1',
 '14dg2aa',
 '14rg10cb',
 '14rg11cb',
 '14rg6b',
 '14rg7b',
 '14rg8bb',
 '16a',
 '16cb',
 '16cl',
 '16rg5aa',
 '20aa1c',
 '20rg1',
 '22rg1ab',
 '22rg2ab',
 '26rg4aa',
 '36aa2',
 '36rg2aa',
 '37aa2',
 '37ac2',
 '37rg1ac',
 '37rg2ac',
 '37rg3ac',
 '38dg1aa',
 '38rg1aa',
 '39aa1',
 '39aa2',
 '39rg1aa',
 '39rg2aa',
 '39rg3aa',
 '39rg4aa',
 '40aa2',
 '40rg1aa',
 '40rg2aa',
 '40rg3aa',
 '41aa1',
 '42aa1',
 '42aa2',
 '42aa3',
 '43ab1',
 '43ab1e',
 '43rg1aa',
 '43rg2aa',
 '43rg3aa',
 '5rg3a',
 '5rg8c',
 '8rg6b',
 '8rg7b',
 '8rg8b',
 '9aa3',
 '9rg2aa',
 '9rg3aa',
 '9rg4aa'}

In [106]:
set(ww_not_H0000['VvN']).difference(set(dt['VvN-code']))

{'10aa1b',
 '10aa2a',
 '10aa2b',
 '11aa3a',
 '11aa3b',
 '11ba1a',
 '11ba1b',
 '11ba2a',
 '11ba2b',
 '11rg2p',
 '12ba1',
 '12ba2',
 '12ba4a',
 '12ba4b',
 '14aa1a',
 '14aa1b',
 '14aa2a',
 '14aa2b',
 '14bb1',
 '14bb2',
 '14bc1a',
 '14bc1b',
 '14bc2a',
 '14bc2b',
 '14ca1a',
 '14ca1b',
 '14ca1c',
 '14ca2a',
 '14ca2b',
 '14ca2c',
 '14ca3a',
 '14ca3b',
 '14cb1a',
 '14cb1b',
 '14cb1d',
 '14cb2a',
 '14cb2b',
 '14dg2',
 '14rg10',
 '14rg11',
 '14rg6',
 '14rg7',
 '14rg8',
 '14rg9',
 '16aa',
 '16aa1a',
 '16aa1b',
 '16aa1c',
 '16aa1d',
 '16ab',
 '16ab2',
 '16ab3a',
 '16ab3b',
 '16ba1a',
 '16ba1b',
 '16ba1c',
 '16bb1a',
 '16bb1b',
 '16bb1c',
 '16bb1d',
 '16rg5',
 '17aa1',
 '20aa1a',
 '22aa1a',
 '22aa1b',
 '22ab1a',
 '22ab1b',
 '22rg1',
 '22rg2',
 '23ab1a',
 '23ab1b',
 '26aa1a',
 '26aa1b',
 '26aa1c',
 '26ab1a',
 '26ab1b',
 '26ac1a',
 '26ac1b',
 '26rg4',
 '27aa1a',
 '27aa1b',
 '27aa2',
 '28aa1a',
 '28aa1b',
 '28aa2a',
 '28aa2b',
 '28aa2c',
 '28aa4a',
 '28aa4b',
 '29aa2a',
 '29aa2b',
 '29aa3',
 '31ab1',

#### Overlap SBB en definitietabel
Alle (op één na) waardes voor SBB uit de definitietabel worden teruggevonden in de waswordt. andersom is dat niet het geval, omdat alleen SBB-codes in de definitietabel staan waarvoor geen VvN code bestaat.

In [71]:
# Aantal unieke waardes van SBB in de was-wordt.
len(set(ww_not_H0000['SBB-code']))

555

In [72]:
# Aantal unieke waardes van SBB in de definitietabel.
len(set(dt['SBB-code']))

67

In [73]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww_not_H0000['SBB-code']).intersection(set(dt['SBB-code'])))

66

In [74]:
set(dt['SBB-code']) - (set(ww_not_H0000['SBB-code']).intersection(set(dt['SBB-code'])))

{'26-xxx [08-f]'}